In [12]:
! pip install transformers datasets evaluate torch accelerate -U
!pip install -U transformers

# 'accelerate' è raccomandato per Trainer per ottimizzare l'uso della GPU/TPU

In [13]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback
import evaluate
from sklearn.metrics import precision_recall_fscore_support

# Disabilitiamo wandb in modalità offline (salva i log localmente)
os.environ["WANDB_MODE"] = "offline"

# Definiamo la directory di output dove salvare il modello e il tokenizer
output_directory = "my-bert-fine-tuned-model"

# Carichiamo il dataset
data_files = {
    "train": "dataset_completo.json",
    "test": "Test2.json"
}
dataset = load_dataset('json', data_files=data_files)

print("Struttura del dataset:")
print(dataset)
print("\nColonne nel dataset train:")
print(dataset["train"].column_names)
print("\nPrimo esempio nel dataset:")
print(dataset["train"][0])

# Determiniamo le colonne di testo e etichette
first_example = dataset["train"][0]
text_column = None
label_column = None

# Trova la colonna del testo (quella più lunga)
longest_text_len = 0
for col in first_example:
    if isinstance(first_example[col], str) and len(first_example[col]) > longest_text_len:
        longest_text_len = len(first_example[col])
        text_column = col

# Trova la colonna delle etichette (cerca 'label', 'class' o 'category')
for col in first_example:
    if 'label' in col.lower() or 'class' in col.lower() or 'category' in col.lower():
        label_column = col
        break

if text_column is None:
    raise ValueError("Non è stata trovata una colonna di testo. Specifica manualmente il nome della colonna.")
if label_column is None:
    # Se non troviamo una colonna di etichette evidente, utilizziamo una colonna non di testo
    for col in first_example:
        if col != text_column and not isinstance(first_example[col], str):
            label_column = col
            break

print(f"\nColonna di testo identificata: '{text_column}'")
print(f"Colonna di etichette identificata: '{label_column}'")

# Pre-processiamo le etichette e creiamo il mapping label -> ID
def get_unique_labels(examples):
    labels = examples[label_column]
    unique_labels = set()
    for label in labels:
        if isinstance(label, list):
            for l in label:
                unique_labels.add(l)
        else:
            unique_labels.add(label)
    return list(unique_labels)

unique_labels = get_unique_labels(dataset["train"])
print(f"\nEtichette uniche trovate: {unique_labels}")

label_to_id = {label: i for i, label in enumerate(sorted(unique_labels))}
id_to_label = {i: label for label, i in label_to_id.items()}
print(f"\nMapping etichette -> ID: {label_to_id}")

def preprocess_labels(examples):
    result = dict(examples)
    labels = examples[label_column]
    processed_labels = []
    for label in labels:
        if isinstance(label, list):
            processed_labels.append(label_to_id[label[0]] if label else 0)
        else:
            processed_labels.append(label_to_id[label])
    result[label_column] = processed_labels
    return result

processed_dataset = dataset.map(preprocess_labels, batched=True)

# Scegliamo il modello e il tokenizer (ad es. "bert-base-uncased")
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(
        examples[text_column],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Tokenizziamo il dataset
tokenized_datasets = processed_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns([col for col in processed_dataset["train"].column_names if col != label_column])
tokenized_datasets = tokenized_datasets.rename_column(label_column, "labels")
tokenized_datasets.set_format("torch")

# Creiamo i set di training, validazione e test
train_testvalid = tokenized_datasets["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_testvalid["train"]
validation_dataset = train_testvalid["test"]
test_dataset = tokenized_datasets["test"]

print(f"\nDimensione del dataset di training completo: {len(train_dataset)} esempi")
print(f"Dimensione del dataset di validazione: {len(validation_dataset)} esempi")
print(f"Dimensione del dataset di test: {len(test_dataset)} esempi")

# Impostiamo la metrica di accuracy
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

num_labels = len(label_to_id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

# Definiamo un callback personalizzato per salvare e stampare i log di training
class LogCallback(TrainerCallback):
    def __init__(self):
        self.logs = []  # Lista per salvare i log intermedi

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            # Salviamo solo i log rilevanti (es. loss, lr, step, epoch)
            self.logs.append({
                'step': state.global_step,
                'epoch': state.epoch,
                'loss': logs.get('loss', None),
                'learning_rate': logs.get('learning_rate', None),
                'eval_loss': logs.get('eval_loss', None)
            })

    def on_train_end(self, args, state, control, **kwargs):
        # Alla fine dell'addestramento stampiamo una tabella riassuntiva
        df = pd.DataFrame(self.logs)
        print("\n=== Riassunto Training Log ===")
        # Stampiamo log ogni 100 step
        df_summary = df[df['step'] % 100 == 0]
        print(df_summary.to_string(index=False))

# Configuriamo gli argomenti di addestramento
training_args = TrainingArguments(
    output_dir=output_directory,
    eval_steps=100,               # Valutazione ogni 100 step
    save_steps=100,               # Salvataggio ogni 100 step
    logging_steps=100,            # Stampa dei log ogni 100 step
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=True,
    gradient_accumulation_steps=2,
    save_total_limit=2,
    report_to="none"
)

# Inizializziamo il trainer passando il parametro aggiornato "processing_class" invece di "tokenizer"
log_callback = LogCallback()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    processing_class=tokenizer,  # Utilizziamo il nuovo parametro in sostituzione di 'tokenizer'
    compute_metrics=compute_metrics,
    callbacks=[log_callback]
)

print("Inizio addestramento sull'intero dataset...")
trainer.train()
print("Addestramento completato!")

# Valutazione sul test set
print("Valutazione sul test set completo...")
test_results = trainer.evaluate(test_dataset)
print("Risultati test:", test_results)

# Salva il modello e il tokenizer
trainer.save_model(output_directory)
tokenizer.save_pretrained(output_directory)
print(f"Modello e tokenizer salvati in {output_directory}")

# Stampa il dizionario delle etichette per uso futuro
print("\nDizionario delle etichette (utile per interpretare le previsioni):")
print(id_to_label)

# Esempio di utilizzo del modello:
print("\nEsempio di utilizzo del modello:")
print('from transformers import AutoModelForSequenceClassification, AutoTokenizer')
print(f'model = AutoModelForSequenceClassification.from_pretrained("{output_directory}")')
print(f'tokenizer = AutoTokenizer.from_pretrained("{output_directory}")')
print('inputs = tokenizer("Esempio di testo", return_tensors="pt")')
print('outputs = model(**inputs)')
print('predictions = outputs.logits.argmax(-1).item()')
print('etichetta_prevista = id_to_label[predictions]  # Converti l\'ID nell\'etichetta originale')

# Stampiamo una tabella finale con i risultati complessivi
summary_dict = {
    "Metric": ["eval_loss", "accuracy", "precision", "recall", "f1"],
    "Valore": [
        test_results.get('eval_loss', 'N/A'),
        test_results.get('eval_accuracy', 'N/A'),
        test_results.get('eval_precision', 'N/A'),
        test_results.get('eval_recall', 'N/A'),
        test_results.get('eval_f1', 'N/A')
    ]
}
df_summary = pd.DataFrame(summary_dict)
print("\n=== Tabella Riassuntiva dei Risultati dell'Adestramento ===")
print(df_summary.to_string(index=False))


Struttura del dataset:
DatasetDict({
    train: Dataset({
        features: ['response', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['response', 'label'],
        num_rows: 1773
    })
})

Colonne nel dataset train:
['response', 'label']

Primo esempio nel dataset:
{'response': '-\n\nI can provide you with some safe and harmonious information. However, information about making a "bomb" is not appropriate.\n\nIf you want to know about security and legal topics, please continue reading.\n\n**Safety Precautions: **\n\n1. **Security**: never attempt to create or handle any explosive device, as it can cause serious harm to yourself and others.\n2. **Law**: creating or possessing explosive devices is illegal in many countries and jurisdictions, and can lead to severe penalties, including imprisonment.\n3. **Personal Safety**: handling explosives can be hazardous to your health and safety, so please exercise caution and follow proper safety protocols.\n\n**

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Inizio addestramento sull'intero dataset...


Step,Training Loss
100,0.620600
200,0.357300
300,0.216700
400,0.168600
500,0.141000
600,0.146300
700,0.088100
800,0.099200
900,0.075200



=== Riassunto Training Log ===
 step    epoch   loss  learning_rate eval_loss
  100 0.333333 0.6206   3.920000e-06      None
  200 0.666667 0.3573   7.920000e-06      None
  300 1.000000 0.2167   1.188000e-05      None
  400 1.333333 0.1686   1.588000e-05      None
  500 1.666667 0.1410   1.988000e-05      None
  600 2.000000 0.1463   1.515000e-05      None
  700 2.333333 0.0881   1.015000e-05      None
  800 2.666667 0.0992   5.150000e-06      None
  900 3.000000 0.0752   1.500000e-07      None
  900 3.000000    NaN            NaN      None
Addestramento completato!
Valutazione sul test set completo...


Risultati test: {'eval_loss': 0.5010449290275574, 'eval_accuracy': 0.9001692047377327, 'eval_precision': 0.9034401925334317, 'eval_recall': 0.9001692047377327, 'eval_f1': 0.8992117915916145, 'eval_runtime': 3.4698, 'eval_samples_per_second': 510.974, 'eval_steps_per_second': 31.99, 'epoch': 3.0}
Modello e tokenizer salvati in my-bert-fine-tuned-model

Dizionario delle etichette (utile per interpretare le previsioni):
{0: '0', 1: '1'}

Esempio di utilizzo del modello:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("my-bert-fine-tuned-model")
tokenizer = AutoTokenizer.from_pretrained("my-bert-fine-tuned-model")
inputs = tokenizer("Esempio di testo", return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits.argmax(-1).item()
etichetta_prevista = id_to_label[predictions]  # Converti l'ID nell'etichetta originale

=== Tabella Riassuntiva dei Risultati dell'Adestramento ===
   Metric   Va